<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear AC 04 - The frequency response

This document deals with the **frequency response** that enables us to obtain the response of a circtuit to any periodic signal.

Version 1.0 (11/5/2019)  
License information is at the end of the document

---
**Bill Of Materials (BOM):**

* Two capacitors of $10nF$ and $100nF$
* Two $47k \Omega$ Resistors

---

# Periodic signals

Peridic signals are signals that repeat themselves over time.

![Periodic Signal](images\Linear_AC_04\periodic_signal.png)

The time it takes the signal to repeat is the **period** and is usually associated to the $T$ letter. Formally, we can define the **period** $T$ of a signal $x(t)$ as the minimum $T$ value that satisfies:

$$x(t)=x(t+n \cdot T)$$

for any integer, positive or negative, $n$ number.

We can define **frequency**, usually associated to the letter $f$, as the inverse of the period:

$$f=\frac{1}{T}$$

As time has **seconds** ($s$) units, frequency has $s^{-1}$ units.  
As the frequency magnitude is used a lot, its unit has its on name **Hertz ($Hz$)**

$$Hz=\frac{1}{s}$$

Frequency is very important in sound because our ears are very sensitive to the frequency of the air pressure waves. In music, the [notes](https://en.wikipedia.org/wiki/Musical_note) are related to the frequency of the sound waves and, each time the frequency **doubles** we go up one [octave](https://en.wikipedia.org/wiki/Octave).

Of course, no signal is trully periodic as it should go from time $-\infty$ to time $+\infty$. In general we analyze **ideal** periodic signals because they are easier to deal with. After we understand them, we can deal with the fact that **real** signals are not trully periodic.

Frequency is important because it can be related to information. Let's consider a signal that can be sent to a far place to send information. Suppose also that it can have two levels and it has a period of $1 \mu s$. Its frequency will be $1MHz$

![Square Wave](images\Linear_AC_04\square_wave.png)

This signal, called a **square** signal, as is trully periodic, does not contain any information. It is always the same. But, if a system can transmit without degradation the above signal it can usually also transmit any similar non periodic signal as long as it does not change faster than the periodic one.

![NP Square](images\Linear_AC_04\np_square.png)

This second signal contains information because each $\mu s$ it can change its value and we don't know, in advance, the value it will take. As each $\mu s$ we can have two equally probable outcomes, we get one [bit](https://en.wikipedia.org/wiki/Bit) of information each $\mu s$, so our information transmision capacity is:

$$\frac{1 bit}{1 \mu s} = 10^6 \frac{bit}{s} = 1 M \frac{b}{s}$$

The bigger the signal frequency a system can transmit, the bigger the information we can send.

Note that any combination of bits is possible, so, in order to be capable to transmit $1Mb/s$ the system shall be able to transmit not only a $1MHz$ signal but also any signal with less than $1MHz$ down to constant DC as long as it has only two **high** and **low** levels.

So, in the end, understanding the capacity of a system to transmit periodic signals is important for **sound** and, in general, for any information transmission system.

Ok, let's try some periodic signals in a linear circuit.

---

![Practical Icon](images/pt.png)

---

## SLab Setup

First we will need to import the **SLab** module

In [ ]:
# Import the main SLab module
import slab

Then we need to connect with the board.

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

It is always a good idea to check the board operation.
The following cell checks the calibration for the DAC channels and the first four ADC channels.

In [ ]:
# Check the calibration
slab.checkCalibration(pause=False,na=4,nm=10)

## Unipolar woes

Mount the following **linear** circuit and connect it to the **SLab** board.

![Circuit](images\Linear_AC_04\circuit.png)

You should remember it from the previous tutorial although this is not exactly the same circuit. One problem associated to the **SLab** system is that it is **unipolar**, it can only generate or read **positive** signals.

In order to work with **negative** signals we need to create our own **ground** instead of using the **SLab ground**. We will use **DAC 2** set to $1.5V$ for that. Fron now, to the end of this tutorial, our ground will be the $1.5V$ output of **DAC1**. Measured and generated voltages use the **SLab ground** as reference, the voltages $V_x'$ respect to our **ground refernce** will be different than the measured voltages $V_x$ respect to the **SLab ground reference**.

$$V_x' = V_x - V_{DAC2}$$

So we will need to substract the **DAC2** voltage to any voltage we measure.

In a similar way, when we use **DAC1** to inject a signal on our circuit, the signal in our new reference $V_{DAC1}'$ can be calculated:

$$V_{DAC1}' = V_{DAC1} - V_{DAC2} = V_{DAC1} - 1.5V$$ 

So, the value we will need to set when using the **SLab reference** $V_{DAC1}$ will be:

$$V_{DAC1} = V_{DAC1}' + V_{DAC2} = V_{DAC1}' + 1.5V$$

## Response against periodic signals

After all explanations in how to turn an **unipolar** system in a **bipolar** one, now it is time to perform some measurements.

Now we will apply a **square** signal and see how the circuit responds.

As, for now, we only want to see the response to **periodic** signals, we will start the measurement after 200 waves have been generated on **DAC1**.

In [ ]:
# Set DAC2 to 1.5V
slab.setVoltage(2,1.5)

# Load a 50 point square wave with 1V low value and 2V high value
slab.waveSquare(1.0,2.0,50)

# Set wave frequency of 1kHz
slab.setWaveFrequency(1000)
# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Generate and plot the wave and return the obtained data
# We will start the measurement after 200 waves have been generated
data=slab.wavePlot(200,returnData=True)

The above plot, as is generated by a **SLab** command, uses the **SLab ground** reference. In order to use our new reference we need to execute the following code cell:

In [ ]:
t    = data[0]           # Get the time vector
vref = data[3]           # Get the reference DAC2 vector in SLab reference
vi   = data[1] - vref    # Compute vi (ADC1) in the new reference
vo   = data[2] - vref    # Compute vo (ADC2) in the new reference

# Plot vi and vo in 
slab.plot1n(t,[vi,vo],'Response to square','time [s]','vi, vo [V]',['vi','vo'])

You can see that the output signal has the same frequency as the input but different shape.

Let's try with another signal. This time a **triangle** one.

This time, we will only show the signals in our $1.5V$ reference. We will also define a function to do all the repetitive plotting.

In [ ]:
# Define a function to do the plotting
def plotRef15(data,title):
    t    = data[0]           # Get the time vector
    vref = data[3]           # Get the reference DAC2 vector in SLab reference
    vi   = data[1] - vref    # Compute vi (ADC1) in the new reference
    vo   = data[2] - vref    # Compute vo (ADC2) in the new reference
    # Plot vi and vo in 
    slab.plot1n(t,[vi,vo],title,'time [s]','vi, vo [V]',['vi','vo'])

# VDAC2 should be already at 1.5V    
    
# Load a 50 point triangle wave with 1V low value and 2V high value
slab.waveTriangle(1.0,2.0,50)

# Set wave frequency of 1kHz
slab.setWaveFrequency(1000)
# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Get measurements without plotting
# We will start the measurement after 200 waves have been generated
data=slab.waveResponse(200)

# Plot results in our 1.5V reference
plotRef15(data,'Response to triangle')

You should also see that the output, again, has the same frequency but also different shape. Note that although in both cases we have outputs with the same frequency as the input, it is dangerous to generalize this result as we will see later.

Now, let's try a **sine** signal.

In [ ]:
# Load a 50 point sine wave with 1V low value and 2V high value
slab.waveSine(1.0,2.0,50)

# Set wave frequency of 1kHz
slab.setWaveFrequency(1000)
# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Get measurements without plotting
# We will start the measurement after 200 waves have been generated
data=slab.waveResponse(200)

# Plot results in our 1.5V reference
plotRef15(data,'Response to 1kHz sine')

This is interesting. The output not only has the same frequency as the input but also the same shape. You can try again using a different **sine** frequency.

In [ ]:
# Load a 50 point sine wave with 1V low value and 2V high value
slab.waveSine(1.0,2.0,50)

# Set wave frequency of 100Hz
slab.setWaveFrequency(100)
# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Get measurements without plotting
# We will start the measurement after 200 waves have been generated
data=slab.waveResponse(200)

# Plot results in our 1.5V reference
plotRef15(data,'Response to 100 Hz sine')

Again, we conserve both the shape and the frequency. Note that the **amplitude** and **phase** changes compared to the previous measurement.

## It's all about exponentials

Conserving the shape is not strange. We know that exponentials are autofunctions of **Linear Time- Independent (LTI)** systems, so, when we enter an exponential $x(t)$ to an **LTI** system we get an output $y(t)$ with the same exponential, multiplied by a constant, on the output.

$$x(t) \overset{LTI}{\longrightarrow} y(t)$$

$$e^{a \cdot t} \overset{LTI}{\longrightarrow} B \cdot e^{a \cdot t}$$

As we know from the [Euler Formula](https://en.wikipedia.org/wiki/Euler%27s_formula), **sine** functions are composed of two exponentials with **imaginary** exponents. So it is normal for them to conserve the shape when going through a **LTI** system. 

Let's consider that we enter a sine $x(t)$ to a **LTI** system:

$$x(t) = sin(\omega t) = \frac{e^{j \omega t}-e^{-j\omega t}}{2j} 
=  \frac{e^{j \omega t}}{2j} - \frac{e^{-j\omega t}}{2j}$$

The output $y(t)$ will get each exponential by a constant:

$$y(t) = z_1 \frac{e^{j \omega t}}{2i} - z_2 \frac{e^{-j \omega t}}{2j}$$

And we chose $z_1$ and $z_2$ so that:

$$z_1 = a + jb \qquad z_2 = a - jb$$

We need to do that so that the output is a real number. More about that later.  

Then we get:

$$y(t) = (a + jb)\frac{e^{j \omega t}}{2j} - (a - jb) \frac{e^{-j \omega t}}{2j}$$

We get:

$$y(t) = a \frac{e^{j \omega t}-e^{-j \omega t}}{2j} 
+ b \frac{e^{j \omega t}+e^{-j \omega t}}{2}$$

And that is equal to:

$$y(t) = a \cdot sin(\omega t) + b \cdot cos(\omega t)$$

And that is equal to:

$$y(t) = \sqrt{a^2+b^2} sin(\omega t+tan^{-1}\frac{a}{b}) = D \cdot sin(\omega t+ \theta)$$

With:

$$D = \sqrt{a^2+b^2} \qquad \theta = tan^{-1}\frac{a}{b}$$

So when we multiply the two exponentials that compose a **sine** wave by two constants, we get a **sine wave** with the same **frequency** although it can have different **amplitude** and **phase**.

Rememeber the two constants we are using:

$$z_1=a+jb \qquad z_2=a-jb$$

The two  constants need to be **complex conjugate**  to each other so that when a **Real** sine enters the system, the output is also **Real** with no imaginary components.  

The **complex conjugate** is associated to the $^*$ unary operator:

$$(a+jb)^* =  a-jb$$

As **LTI** systems don't generate imaginary outputs, the constants will always be **complex conjugates** to each other and we can always deduce one constant from the other just by changing the sign of the imaginary component.

## Response to non sinusoidal periodic signals

We explained in previous documents that any signal can be obtained by adding exponentials together. In fact, that gives us the **Laplace** transform. In the case of **periodic** signals, if we add exponentials to obtain a **periodic** signal, the exponentials shall also be **periodic**. And the only exponentials that are periodic are the ones that feature a full imaginary exponent without any real component. 

$$z = j\omega$$

Note that we are not talking about how many exponentials we shall add to match a particular signal. The number of exponentials to add to obtain a perfect match of, for instance, a square wave, could be infinite. The [Fourier Series](https://en.wikipedia.org/wiki/Fourier_series) give just that: a series of sine waves that compose any particular periodic signal.

In the end, any **periodic** signal can be obtained as the sum of **sine** waves with different **frequencies**, **amplitudes** and **phases**. The frequency of the **sine** waves are restricted because only full sine waves can be included in one period of the original signal. That means that the frequency of the sine waves need to be an integer multiple of the original wave frequency. 

$$f = n\cdot f_0$$

Joining all the previous explanations we can see that any periodic signal can be created just adding sine waves. We also know that those sine waves don't change shape nor frequency when going through a **LTI** system, they only change in **amplitude** and **phase**.

So, we can obtain the response of any **LTI** system to periodic signals by knowing its response to sine waves of different frequencies. This response to sine waves is called the **Frequency Response** of the **LTI** system.


## Measuring the frequency response

To measure the frequency response of a **LTI** system we just need to obtain how sinewaves of any frequency change in amplitude and phase.

We will use the already mounted circuit to measure its response to a $1000 Hz$ sine wave. In fact we have done that before. But this time, instead of starting the measurement after 200 sine waves have been generated, we will measure just from the beginning.

In [ ]:
# Set wave frequency of 1000Hz
slab.setWaveFrequency(1000)
# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Set storage for 250 samples of ADC1, ADC2 and ADC3 (5 waves)
slab.setTransientStorage(250,3)

# Get measurements without plotting
# We will start the measurement at time zero
data=slab.waveResponse()

# Plot results in our 1.5V reference
plotRef15(data,'Response to 1 kHz sine')

The input signal is not really periodic as we apply it only from time $t=0$. As you can see you have two time phases in the output:

* In the first **transient phase** the sine wave changes with time

* In the second **stationary phase** the sine wave always feature the same amplitude and phase

Only the second **stationary phase** is the response to a **periodic** sine wave. The first phase is only a transition due to the fact that the input is not trully periodic. Note that, as the first phase does not repeat, we cannot really talk about **frequency** in this phase.

You can instruct **SLab** to apply several periods of the input wave, as we have done before, before starting the measurements. That way you can skip the **transient phase** and see only the **stationary phase**.

In [ ]:
# Get measurements without plotting
# We will start the measurement after 10 sine waves
data=slab.waveResponse(10)

# Plot results in our 1.5V reference
plotRef15(data,'Response to 1 kHz sine')

The **SLab AC module** includes functions to extrat the amplitude an phase information from sinewave measurements. In particular the **sineGain** function obtain the **complex gain** of the circuit against a sine wave.

$$gain = \frac{y(t)}{x(t)}$$

This is a **complex gain** its **modulus** give the **Amplitude gain** and its  **phase** gives the **phase change**.

The following code cell imports the **AC module** and uses it to obtain the **complex gain** of our circuit agains a $100Hz$ sine wave.

In [ ]:
# Import the slab AC module
import slab.ac as ac

# Compute the complex gain
# Sine wave goes from 1.5V to 2V and has 1000Hz frequency
# We measure output on channel 2 
# We apply 10 input wave cycles before measuring
gain = ac.sineGain(1.0,2.0,1000,2,10)

# Show the complex gain
print('Gain at 1000Hz =',gain)

It is difficult to make sense of a **complex gain** so let's convert it to **Amplitude gain** and **Phase change**

In [ ]:
# Convert complex to modulus and phase
AGain,phase = ac.magPhase(gain)

# Show it
print('Amplitude gain =',AGain)
print('Phase change =',phase)

You can see that the output is about 1/3 of the input and that it lags about 70 degrees. The output is close to be in [quadrature](https://en.wikipedia.org/wiki/In-phase_and_quadrature_components) respect to the input.

You can check that the numbers are right by excecuting the code below.

This code compares the measured output signal agains a sinewave that has the obtained **Amplitude gain** and **Phase change**.

In [ ]:
# Impor the numpy module to use some of its functions
import numpy as np

# Extract measurement information
t    = data[0]           # Get the time vector
vref = data[3]           # Get the reference DAC2 vector in SLab reference
vo   = data[2] - vref    # Compute vo (ADC2) in the new reference

w  = 2*np.pi*1000        # Compute frequency in rad/s
ph = phase*np.pi/180.0   # Compute phase in radians

# Generate a signal with the calculated amplitude and phase changes
vo2 = 0.5*AGain*np.sin(w*t+ph)

# Plot the measured signal against the calculated signal
slab.plot1n(t,[vo,vo2],'','time [s]','vo [Volt]',['Measured','Calculated'])

If you measure the **stationary phase** for several frequencies you can plot how the amplitude and phase change at each frequency. We can use the **AC module** **freqResponse** function for that.

In [ ]:
# Define a vector with a list of frequencies to measure
fvector = np.linspace(10,3000,40)

# Perform the measurement at each frequency an get its complex gain
# Input signal in DAC1 will go from 1V to 2V
# We will measure on ADC2 and wait 20 cycles before measuring
g = ac.freqResponse(1.0,2.0,fvector,2,20)

# Plot the complex gain in modulus and phase

ac.plotFreq(fvector,g)

And this is the frequency response of our system.

You can see that, at $1000 Hz$ we get the same result we have obtained before.

Note that the phase is negative because it always lags the input. It is an expected result. The output shall be [causal](https://en.wikipedia.org/wiki/Causality), so an **LTI** shall always respod **after** an input chanfe, never **before** it.

## Calculating the frequency response

Measuring the **frequency response** is quite easy. Obtaining that from a circuit description, by performing calculations, is a different thing.

As we know, any **LTI** system can be described by its **Impulsional Response** $h(t)$. The response to a sine wave can be obtained by convolution of the **Impulsional Response** and the sine wave:

$$y(t) = h(t) * sin(\omega \cdot t)$$

But we can do that also in the **Laplace S domain:**

$$Y(s) = H(s) \: \mathcal{L}(sin(\omega \cdot t))$$

As we remember, $H(s)$ **poles** correspond to **exponentials** in the time domain. Poles in the $i\omega$ axis correspond to sinewaves whereas poles with **negative real** values correspond to decaying exponentials. Poles with **positive real** values correspond to unstable systems so we won't deal with them.

The **Laplace** transform of a sine wave is:

$$sin(\omega \cdot t) \: u(t) \overset{\mathcal{L}}{\longrightarrow} \frac{\omega}{s^2+\omega ^2}$$

As you can see it has two poles in the imaginary axis.

$$s^2+\omega ^2=0 \quad \rightarrow \quad s = \pm j\omega$$

And you can separate them in two fractions:

$$\frac{\omega}{s^2+\omega ^2}
= \frac{1/2j}{s-jw} - \frac{1/2j}{s+jw}$$

When we multiply $H(s)$ and the transform of the sine waves we obtain for $Y(s)$ all the poles of $H(s)$ and the two new **imaginary** poles assosiated to the sine wave. We could now use the **Inverse Laplace** transform to obtain $y(t)$.

This could be quite complicated so we won't do that. We won't do that  because we don't need $y(t)$ with the **transient** and **stationary** phases. We only need the last **stationary** phase.

$H(s)$ is a polynomial fraction. By factoring $H(s)$ in all its **poles** (denominator **roots**) you get one exponential in the time domain for each pole. Each pole will relate to one exponential in the time domain.

$H(s)$ don't have poles with positive real values or the system won't be stable. Poles with negative real values relate to exponentials that will decay over time, so they are part of the **transient** phase. We won't consider, for now, $H(s)$ functions with poles in the **imaginary** axis. Those functions correspond to **linear oscilators** and are a matter of another tutorial.

In the end, the only poles that will remain in the **stationary** phase of $Y(s)$ are the ones in the **imaginary** axis and those are the ones associated to the input **sine** wave. All poles in $H(s)$ have negative **real** components and will decay inside the **transient phase**. 

The two **complex poles** poles that remain in the **stationary phase (ST)** will be affected by the value of $H(s)$ at their position, however:

$$Y_{ST}(s) = H(jw)\frac{1/2j}{s-jw} - H(-jw)\frac{1/2j}{s+jw}$$

You can argue that the above functions are defined for all the **s plane** not just for the $\pm \omega$ pole positions and you should use $H(s)$ instead. This is true, but those functions correspond to exponentials that are zero for negative times, for periodic signals, only the asymptote is important.

Note also that in every **LTI** system $H(-j\omega)$ is the complex conjugate of $H(j\omega)$, so we can obtain:

$$Y_{ST}(s) = H(jw)\frac{1/2j}{s-jw} - H(jw)^*\frac{1/2j}{s+jw}$$

Returning to the time domain is easy because we have the same poles associated to a sine wave that we started with. The only difference is the $H(s)$ multiplication at the position of the poles that generate a change in **amplitude** and **phase** on the sinewave.

$$y_{ST}(t) = H(jw) \: \frac{e^{j \omega t}}{2j} - H(jw)^* \: \frac{e^{-j \omega t}}{2j}$$

We have done that before. We know that this is gives an **Amplitude gain** equal to the modulus of $H(jw)$ and a phase change equal to the phase of $H(jw)$.

## Circuit's frequency response

Let's apply that to our circuit. In order to obtain $H(s)$ we will sustitute each linear component by its **s impedance**.

![Circuit Z](images\Linear_AC_04\circuit3z.png)

Note that using **DAC2** to generate a **new ground reference** does not change the behavior of the circuit.

We have dealt with this circuit in the previous tutorial, so we known its $H(s)$ **Transfer function**:

$$H(s)=\frac{R_1 \; C_1 \; s}
{1+s(R_2 \; C_1 + (R_1+R_2)C_2) + C_1 \; C_2 \; R_1 \; R_2 \; s^2}$$

And we know that it features one **zero** and two **poles**:

$$s_z = 0 \qquad s_{p1} = -192 \qquad s_{p1} = -2361$$ 

We can plot the **modulus** of the **Transfer function** $H(s)$ in the **s plane**

In [ ]:
import numpy as np  # Import the numpy module

# Component values in Ohm and Farads
R1 = 47000  
R2 = 47000
C1 = 100E-9
C2 = 10E-9

# 3D function to plot
def H(x,y):
    z = x+y*1j
    return R1*C1*z/(1+z*(R2*C1+(R1+R2)*C2)+C1*C2*R1*R2*z*z)

# Modulus (clipped at 10)
slab.plotFunc3D(H,(-3000,0),(-1500,1500),'','$\sigma$','$\omega$','H(s) Modulus'
                 ,clip=(-10,10),fpost=np.absolute)

In the figure you should be able to se the **zero** and the two **real poles**.

As the most intereting region is the **imaginary axis**, lets rotate the plot to see it better. Also we will change the z limits and the plot range.

In [ ]:
# Modulus (clipped at 5)
# Set an elevation angle of 20 deg and an azimut angle of 10 deg
slab.plotFunc3D(H,(-3000,0),(-6000,6000),'','$\sigma$','$\omega$','H(s) Modulus'
                 ,clip=(-5,5),fpost=np.absolute,angles=(20,10))

As explained before, the **steady phase** response to a sine wave depends only on $H(i\omega)$ not on the rest of the **s plane**. The following code shows the **modulus** and **phase** of $H(i\omega)$. As you can check the modulus can be seen at the front of the 3D plots.

At the start of this chapter we applied a $1000 Hz$ $1V$ sine wave to our circuit. We can now obtain $H(s)$ at this frequency on the **s domain** over the imaginary axis.

$$z = H(j\cdot 1000 Hz \cdot 2 \pi)$$

So, the output sine will have an amplitude gain and phase change:

$$A_G = |z| \qquad \theta = \varphi_z$$

Use the following code cell to calculate the above values:

In [ ]:
w = 1000.0*2.0*np.pi
z = H(0.0,w)
print('z =',z)

# Convert complex to modulus and phase
AGain,phase = ac.magPhase(gain)

# Show it
print('Amplitude gain =',AGain)
print('Phase change =',phase)

So, in the end, if we have a **Transfer function** $H(s)$ that describes one **LTI** system, and we feed it with an input sine periodic signal:

$$x(t) = A \cdot sin(\omega \cdot t)$$

The output of the system can be calculated as:

$$y(t) = |H(j\omega)| \cdot A \cdot sin(\omega \cdot t + \varphi_{H(j\omega)})$$

So, for each angular frequency $\omega$ we get:

* $|H(j\omega)|$ as the amplitude **gain**

* $\varphi_{H(j\omega)}$ as the **phase** change 

We can represent both in two curves. For instance for our example circuit. As we have measured this information previously, we will also compare the calculations with the measurements.

In [ ]:
# Calculation of complex gain from H(s)
g2 = H(0.0,2.0*np.pi*fvector)

# Plot the frequency response both for the measuremnt and the calculations
ac.plotFreq([fvector,fvector],[g,g2],['Measured','Calculated'])

You should get a good match between the measurements and the results obtained from the computed $H(s)$ **Transfer function**. A perfect match is impossible as resistors and capacitors have some tolerances.

Those two curves can be used to calculate how the system responds to any perodic signal.  
Any periodic signal we can decompose it in a series of sine waves.

$$x(t)=\sum_{k=1}^n A_k sin(\omega_k t + \theta_k)$$

Every sine frequency is affected by the **amplitude** and **phase** values  from the curves in the point that correspond to this frequency. After computing how the **LTI** system affects each sinewave we can join the waves again to obtain the final response to the periodic signal.

$$y(t)=\sum_{k=1}^n |H(j\omega_k)| A_k  sin(\omega_k t + \theta_k + \varphi_{H(j\omega)})$$

## Bode plots

## Bode Plots

Signals that we use in **LTI** systems can have a big [dynamic range](https://en.wikipedia.org/wiki/Dynamic_range). Using linear values for the amplitudes make the amplitude curve too close to the horizontal axis when the gain is much lower than the maximum gain. You can see that in our example circuit for frequencies above $6000Hz$

In [ ]:
# Vector or frequencies from 1Hz to 10kHz
fvect3 = np.linspace(1,10000,400)

# Calculation of complex gain from H(s)
g3 = H(0.0,2.0*np.pi*fvect3)

# Plot the frequency response both for the measuremnt and the calculations
ac.plotFreq(fvect3,g3)

In order to **compress** the **amplitude** information we use $dB$ units for gain.

$$gain[dB] = 20 \: log_{10} \: gain[linear] $$

A gain of one equals to zero $dB$. Each time the gain increases 10 times we increase in $20 dB$. So, for instance a gain of $1000$ equals $60 dB$ and a gain of $0.01$, or $1/100$ attenuation, equals $-40 dB$.

It is important to note that $dB$ does not directly relate to any unit like **Volt** or **Ampere** it just relates two magnitudes.

Formally, $dB$ units compare **powers**. In fact, the proper definition of $dB$ is:

$$Power Ratio[dB] = 10 \: log_{10} \frac{P_2}{P_1}$$

In electric circuits, when using a resistive load, power is proportional to the **square** of **voltage** or **current**.

$$P_R = \frac{V_R^2}{R} = I^2 \cdot R$$ 

So, if, we define our gain as voltage gain.

$$gain_V = \frac{V_2}{V_1}$$

The power gain will be:

$$gain_P = \frac{V_2^2}{V_1^2}$$

So, the **dB** power ratio will be.

$$Power Ratio[dB] = 10 \: log_{10} \frac{P_2}{P_1} 
= 10 \: log_{10} \frac{V_2^2}{V_1^2} 
= 20 \: log_{10} \frac{V_2}{V_1} = 20 \: log_{10} \: gain_V$$

Another problem with the frequency response curves is using **linear** horizontal frequency axes. The hearing response in humans is logarithmic. In music, as explained before, we go up one **octave** each time we **double** the frequency. So, for instance 4 **octaves** correspond to a $2^4 = 16$ frequency ratio.

Also using **logarithmic** units for frequency compress the values so, in general, it is good when the maximum frequency to show is several [orders of magnitude](https://en.wikipedia.org/wiki/Order_of_magnitude) over the minimum frequency to show.

So, in general, it is normal to also use a logarithmic scale for frequencies.

In the end, we represent the frequency response using a double logarithmic plot for gains:

* Vertical logarithmic axis due to the use of $dB$ units
* Horizontal logarithmic frequency axis 

The phase plot also uses logarithmic horizontal frequency axis but linear vertical **degree** or **radian** values.

The pair of amplitude log-log and phase log-linear plots compose what is called a **Bode Plot**. The following code shows the bode plot for our circuit.


In [ ]:
# Bode plot from the previous data
ac.plotBode(fvect3,g3)

To obtain a proper bode plot, the data used to generate it should also be spaced in a log space, so, let's recalculate the above plot.

In [ ]:
# Vector or frequencies from 1Hz to 10kHz
fvect3 = ac.logRange(1,10000) 

# Calculation of complex gain from H(s)
g3 = H(0.0,2.0*np.pi*fvect3)

# Bode plot from the previous data
ac.plotBode(fvect3,g3)

That's way better.

Usually the major tick divisions in the frequency axis are marked in **Hz decades**, that is, each time the frequency multiplies by $10$. **Octave** and **radian/s** markings can also be used by they are not so common.

An inconvenient of the **bode plot** is that we cannot locate the **DC** zero frequency point as $log_{10}(0)$ is not a real number.

The bode plot contain all needed information to obtain the response of a **LTI** system to periodic signals in the **stationary** phase. Note, however, that it does not give information on the **transient** phase. For this phase you will need all the $H(s)$ information, not just the $H(i \omega)$ information in the imaginary axis.

## Last words

In this chapter we have dealt with the **frequency response** of AC circuits. This is the base of all frequency selective circuits like [filters](https://en.wikipedia.org/wiki/Filter_(signal_processing)), for instance.

As we have explained, the frequency response also can be related to the capacity of a **LTI** system to transmit information.

This was a long and quite theoretical tutorial so we should leave it now. In the next tutorial we will continue dealing with the **Bode plots** because they feature a lot of information that is very important to understand.

## Document license

Copyright  ©  Vicente Jiménez (2019)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">